In [1]:
import tensorflow as tf
import _init_paths
import os
import numpy as np
from functools import partial
import tensorflow.keras.models as KM
import tensorflow.keras.backend as K
from utils import DisplayPlot
from mydata.loader_coco import COCO
from mymodel.wgangp import WGanGP
from mymodel.srgen import SrGen
from mymodel.utils import Vgg, Resnet, Discriminator
import cfg

In [2]:
import argparse
def GetTrainingOptions(weightAdv = 0.1, # weight for adversal loss
                       weightGP = 10, # weight for gradient Penalty
                       weightFeat = 10, # weight for feature reconstruction loss
                       weightPixel = 0.1, # weight for pixel-wise mean absolute error 
                       devices = '2', # gpu devices to use
                       batch = 12,
                       ratio = 2, # resolution ratio between low and high resolution images
                       lr_g = 5e-5, # learning rate for generator
                       lr_d = 1e-4, # learning rate for discriminator
                       epochs = 100, # epochs to train
                       size_train = 1000, # num of training data per epoch
                       size_val = 1, # num of validation data per epoch
                       dir_results = '.', # root directory for saving results
                       ):
    
    opt = argparse.Namespace()
    opt.weightAdv = weightAdv
    opt.weightGP = weightGP
    opt.weightFeat = weightFeat
    opt.weightPixel = weightPixel
    opt.devices = devices
    opt.batch = batch
    opt.ratio = ratio
    opt.lr_g = lr_g
    opt.lr_d = lr_d
    opt.epochs = epochs
    opt.size_train = size_train
    opt.size_val = size_val
    opt.dir_results = dir_results
    return opt


In [3]:
opt = GetTrainingOptions(epochs=200, weightFeat=20)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = opt.devices

### dataset

In [5]:
ds = COCO(root=cfg.PATH_COCO, ratio=opt.ratio, batch_size=opt.batch)
ds_tr, ds_val = ds.GetDataset()

### Model

In [6]:
generator = SrGen(input_shape=ds.GetShapeLow())
generator.load_weights(os.path.join('save_model', 'weights.h5'))
discriminator = Discriminator(input_shape=ds.GetShapeHigh())
feat = Resnet(input_shape=ds.GetShapeHigh())

### Main

In [8]:
optimizer_g = tf.keras.optimizers.RMSprop(opt.lr_g)
optimizer_d = tf.keras.optimizers.RMSprop(opt.lr_d)
model = WGanGP(discriminator, generator, feat, opt)
model.compile(optimizer_g=optimizer_g, optimizer_d=optimizer_d)
display = DisplayPlot(root='Imgs', ds=ds)

In [ ]:
model.fit(ds_tr,
          steps_per_epoch = opt.size_train,
          epochs = opt.epochs, 
          verbose = 1,
          validation_data = ds_val,
          validation_steps = opt.size_val,
          callbacks = [display])

Epoch 1/200
1000/1000 [==============================] - 183s 183ms/step - loss_d: 2.2669 - loss_g: -0.4228 - gp: 2.6071 - loss_adv: -0.6207 - loss_feat0: 0.0357 - loss_feat1: 0.0667 - loss_feat2: 0.0786 - loss_mae: 0.0169 - val_loss_d: -0.2704 - val_loss_g: 0.9819 - val_gp: 2.3499 - val_loss_adv: 0.8072 - val_loss_feat0: 0.0285 - val_loss_feat1: 0.0612 - val_loss_feat2: 0.0733 - val_loss_mae: 0.0118
Epoch 2/200
1000/1000 [==============================] - 179s 179ms/step - loss_d: 2.2446 - loss_g: -0.1604 - gp: 1.8684 - loss_adv: -0.3838 - loss_feat0: 0.0388 - loss_feat1: 0.0718 - loss_feat2: 0.0886 - loss_mae: 0.0242 - val_loss_d: 3.2784 - val_loss_g: -0.9657 - val_gp: 2.2534 - val_loss_adv: -1.1577 - val_loss_feat0: 0.0334 - val_loss_feat1: 0.0631 - val_loss_feat2: 0.0742 - val_loss_mae: 0.0212
Epoch 3/200
1000/1000 [==============================] - 179s 179ms/step - loss_d: 1.8448 - loss_g: -1.3570 - gp: 1.5612 - loss_adv: -1.5845 - loss_feat0: 0.0389 - loss_feat1: 0.0726 - loss_f

1000/1000 [==============================] - 179s 179ms/step - loss_d: 0.7140 - loss_g: -5.7938 - gp: 0.4031 - loss_adv: -5.9426 - loss_feat0: 0.0200 - loss_feat1: 0.0577 - loss_feat2: 0.0614 - loss_mae: 0.0097 - val_loss_d: 0.3868 - val_loss_g: -6.2924 - val_gp: 0.5944 - val_loss_adv: -6.4347 - val_loss_feat0: 0.0190 - val_loss_feat1: 0.0546 - val_loss_feat2: 0.0585 - val_loss_mae: 0.0102
Epoch 22/200
1000/1000 [==============================] - 180s 180ms/step - loss_d: 0.7107 - loss_g: -5.7768 - gp: 0.4133 - loss_adv: -5.9232 - loss_feat0: 0.0196 - loss_feat1: 0.0571 - loss_feat2: 0.0602 - loss_mae: 0.0095 - val_loss_d: 0.0883 - val_loss_g: -6.1085 - val_gp: 0.1354 - val_loss_adv: -6.2701 - val_loss_feat0: 0.0240 - val_loss_feat1: 0.0605 - val_loss_feat2: 0.0650 - val_loss_mae: 0.0121
Epoch 23/200
1000/1000 [==============================] - 180s 180ms/step - loss_d: 0.7322 - loss_g: -5.9121 - gp: 0.4269 - loss_adv: -6.0568 - loss_feat0: 0.0190 - loss_feat1: 0.0567 - loss_feat2: 0.0

1000/1000 [==============================] - 180s 180ms/step - loss_d: 0.5120 - loss_g: -7.4093 - gp: 0.2963 - loss_adv: -7.5437 - loss_feat0: 0.0166 - loss_feat1: 0.0541 - loss_feat2: 0.0552 - loss_mae: 0.0085 - val_loss_d: 0.5074 - val_loss_g: -8.0217 - val_gp: 0.1331 - val_loss_adv: -8.1670 - val_loss_feat0: 0.0183 - val_loss_feat1: 0.0579 - val_loss_feat2: 0.0600 - val_loss_mae: 0.0090
Epoch 42/200
1000/1000 [==============================] - 180s 180ms/step - loss_d: 0.4825 - loss_g: -7.5149 - gp: 0.2780 - loss_adv: -7.6522 - loss_feat0: 0.0180 - loss_feat1: 0.0545 - loss_feat2: 0.0560 - loss_mae: 0.0088 - val_loss_d: -0.0358 - val_loss_g: -7.4950 - val_gp: 0.1661 - val_loss_adv: -7.6467 - val_loss_feat0: 0.0191 - val_loss_feat1: 0.0536 - val_loss_feat2: 0.0650 - val_loss_mae: 0.0141
Epoch 43/200
1000/1000 [==============================] - 179s 179ms/step - loss_d: 0.4397 - loss_g: -7.6162 - gp: 0.2774 - loss_adv: -7.7518 - loss_feat0: 0.0172 - loss_feat1: 0.0541 - loss_feat2: 0.